In [ ]:
# THIS IS A TEST NOTEBOOK, MIGHT NOT APPLY IT. 

In [39]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('migraine_balanced.csv')

X = df.drop(columns=['Type']).values.astype(np.float32)
y = LabelEncoder().fit_transform(df['Type'])

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

num_features = X_train.shape[1]
num_classes = len(np.unique(y))

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)

        )

    def forward(self, x):
        return self.net(x)

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss {loss.item():.4f}")

print("Training complete.")
# -------------------------
# Evaluate accuracy
# -------------------------

model.eval()
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_train).sum().item()
    total = y_train.size(0)
    train_accuracy = correct / total

print(f"Training accuracy: {train_accuracy*100:.2f}%")
# -------------------------
# Test accuracy
# -------------------------

with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    test_accuracy = correct / total

print(f"Test accuracy: {test_accuracy*100:.2f}%")


Epoch 0 | Loss 1.9460
Epoch 10 | Loss 1.4229
Epoch 20 | Loss 0.5030
Epoch 30 | Loss 0.2411
Epoch 40 | Loss 0.1712
Training complete.
Training accuracy: 94.87%
Test accuracy: 91.62%


In [42]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
class CNN1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(32, 16, kernel_size=3),
            nn.ReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear((num_features-4)*16, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = x.unsqueeze(1)           # batch x 1 x features
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = CNN1D()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss {loss.item():.4f}")

print("Training complete.")
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_train).sum().item()
    total = y_train.size(0)
    train_accuracy = correct / total

print(f"Training accuracy: {train_accuracy*100:.2f}%")
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    test_accuracy = correct / total

print(f"Test accuracy: {test_accuracy*100:.2f}%")

Epoch 0 | Loss 1.9513
Epoch 10 | Loss 1.7552
Epoch 20 | Loss 1.3337
Epoch 30 | Loss 0.8916
Epoch 40 | Loss 0.6355
Training complete.
Training accuracy: 82.86%
Test accuracy: 79.77%


In [46]:
# ============================================
#   IMPORTS
# ============================================
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from pytorch_tabnet.tab_model import TabNetClassifier




# Separate features & target
X = df.drop(columns=["Type"]).values.astype(np.float32)
y = LabelEncoder().fit_transform(df["Type"])


# ============================================
#   NORMALIZE FEATURES
# ============================================
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)


# ============================================
#   TRAIN/TEST SPLIT
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Ensure everything is NumPy (required by TabNet)
X_train = np.asarray(X_train, dtype=np.float32)
X_test = np.asarray(X_test, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)


# ============================================
#   TABNET MODEL
# ============================================
clf = TabNetClassifier()

clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    max_epochs=150,
    patience=20,
    batch_size=256,
    virtual_batch_size=128,
)


# ============================================
#   PREDICTIONS
# ============================================
preds = clf.predict(X_test)


# ============================================
#   EVALUATION
# ============================================
acc = accuracy_score(y_test, preds)
print(f"\nTest Accuracy: {acc*100:.2f}%\n")

print("\nClassification Report:")
print(classification_report(y_test, preds))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))



C:\Users\Laphop\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.31574 | val_0_accuracy: 0.12139 |  0:00:00s
epoch 1  | loss: 1.79629 | val_0_accuracy: 0.14451 |  0:00:00s
epoch 2  | loss: 1.5536  | val_0_accuracy: 0.35549 |  0:00:00s
epoch 3  | loss: 1.34331 | val_0_accuracy: 0.52312 |  0:00:00s
epoch 4  | loss: 1.16578 | val_0_accuracy: 0.56647 |  0:00:01s
epoch 5  | loss: 0.99893 | val_0_accuracy: 0.66763 |  0:00:01s
epoch 6  | loss: 0.86883 | val_0_accuracy: 0.70809 |  0:00:01s
epoch 7  | loss: 0.79181 | val_0_accuracy: 0.71098 |  0:00:01s
epoch 8  | loss: 0.69635 | val_0_accuracy: 0.75434 |  0:00:02s
epoch 9  | loss: 0.60162 | val_0_accuracy: 0.81503 |  0:00:02s
epoch 10 | loss: 0.5483  | val_0_accuracy: 0.82081 |  0:00:02s
epoch 11 | loss: 0.52545 | val_0_accuracy: 0.79191 |  0:00:02s
epoch 12 | loss: 0.50761 | val_0_accuracy: 0.81792 |  0:00:02s
epoch 13 | loss: 0.45729 | val_0_accuracy: 0.83526 |  0:00:03s
epoch 14 | loss: 0.43455 | val_0_accuracy: 0.82948 |  0:00:03s
epoch 15 | loss: 0.37824 | val_0_accuracy: 0.84971 |  0

C:\Users\Laphop\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
